In [ ]:
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

def load_and_test_model():
    """
    Charge le modèle sauvegardé et effectue des tests
    """
    
    # 1. Charger le modèle
    try:
        model = joblib.load('pathloss_predictor.pkl')
        print("✓ Modèle chargé avec succès!")
        print(f"Type de modèle: {type(model)}")
        
        # Afficher les paramètres du modèle (correction ici)
        try:
            params = model.get_params()
            print("Paramètres principaux du modèle:")
            important_params = ['n_estimators', 'max_depth', 'learning_rate', 'subsample', 'colsample_bytree']
            for param in important_params:
                if param in params:
                    print(f"  {param}: {params[param]}")
        except Exception as e:
            print(f"Impossible d'afficher les paramètres: {e}")
        
    except FileNotFoundError:
        print("❌ Erreur: Fichier 'pathloss_predictor.pkl' non trouvé!")
        print("Veuillez d'abord exécuter l'entraînement du modèle.")
        return None
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return None
    
    return model

def test_model_predictions(model):
    """
    Teste le modèle avec des exemples de prédiction
    """
    print("\n=== TESTS DE PREDICTION ===")
    
    # Exemples de test avec différents scénarios
    test_cases = [
        # (num_walls, distance_m, frequency_mhz, description)
        (0, 5.0, 2400, "Très proche, aucun mur (LOS)"),
        (1, 10.0, 2400, "Distance moyenne, 1 mur"),
        (3, 15.0, 2400, "Distance moyenne, 3 murs"),
        (0, 30.0, 2400, "Loin, aucun mur (LOS)"),
        (5, 25.0, 2400, "Loin, 5 murs (NLOS dense)"),
        (2, 10.0, 5000, "Distance moyenne, 2 murs, 5GHz"),
        (1, 8.0, 900, "Proche, 1 mur, 900MHz"),
        (0, 1.0, 2400, "Très très proche"),
        (7, 40.0, 5800, "Très loin, nombreux murs"),
    ]
    
    predictions = []
    
    for num_walls, distance, frequency, description in test_cases:
        try:
            # Créer DataFrame avec les features
            features = pd.DataFrame([[num_walls, distance, frequency]], 
                                  columns=['num_walls', 'distance', 'frequency'])
            
            # Prédiction
            path_loss = model.predict(features)[0]
            predictions.append(path_loss)
            
            print(f"{description:<35}: {path_loss:.2f} dB")
            print(f"  Features: {num_walls} murs, {distance}m, {frequency}MHz")
            print()
        except Exception as e:
            print(f"❌ Erreur lors de la prédiction pour {description}: {e}")
            predictions.append(np.nan)
    
    return test_cases, predictions

def validate_model_logic(test_cases, predictions):
    """
    Valide la logique des prédictions du modèle
    """
    print("=== VALIDATION DE LA LOGIQUE ===")
    
    # Filtrer les prédictions valides
    valid_predictions = [p for p in predictions if not np.isnan(p)]
    if len(valid_predictions) < 3:
        print("❌ Pas assez de prédictions valides pour la validation")
        return
    
    # Test 1: Plus de distance = plus de path loss
    print("1. Test distance croissante (même config):")
    distance_test = [
        (2, 5.0, 2400),   # 5m
        (2, 15.0, 2400),  # 15m  
        (2, 30.0, 2400),  # 30m
    ]
    
    distance_predictions = []
    for walls, dist, freq in distance_test:
        try:
            features = pd.DataFrame([[walls, dist, freq]], 
                                  columns=['num_walls', 'distance', 'frequency'])
            pl = model.predict(features)[0]
            distance_predictions.append(pl)
            print(f"  {dist}m: {pl:.2f} dB")
        except Exception as e:
            print(f"  ❌ Erreur pour {dist}m: {e}")
            distance_predictions.append(np.nan)
    
    # Vérifier que le path loss augmente avec la distance
    valid_dist_pred = [p for p in distance_predictions if not np.isnan(p)]
    if len(valid_dist_pred) >= 2:
        distance_increasing = all(valid_dist_pred[i] <= valid_dist_pred[i+1] 
                                for i in range(len(valid_dist_pred)-1))
        print(f"  ✓ Path loss croît avec distance: {'OUI' if distance_increasing else 'NON'}")
    
    # Test 2: Plus de murs = plus de path loss
    print("\n2. Test nombre de murs croissant (même distance):")
    walls_test = [
        (0, 15.0, 2400),  # 0 mur
        (2, 15.0, 2400),  # 2 murs
        (5, 15.0, 2400),  # 5 murs
    ]
    
    walls_predictions = []
    for walls, dist, freq in walls_test:
        try:
            features = pd.DataFrame([[walls, dist, freq]], 
                                  columns=['num_walls', 'distance', 'frequency'])
            pl = model.predict(features)[0]
            walls_predictions.append(pl)
            print(f"  {walls} murs: {pl:.2f} dB")
        except Exception as e:
            print(f"  ❌ Erreur pour {walls} murs: {e}")
            walls_predictions.append(np.nan)
    
    valid_walls_pred = [p for p in walls_predictions if not np.isnan(p)]
    if len(valid_walls_pred) >= 2:
        walls_increasing = all(valid_walls_pred[i] <= valid_walls_pred[i+1] 
                             for i in range(len(valid_walls_pred)-1))
        print(f"  ✓ Path loss croît avec murs: {'OUI' if walls_increasing else 'NON'}")
    
    # Test 3: Impact de la fréquence
    print("\n3. Test fréquences différentes (même config):")
    freq_test = [
        (2, 15.0, 900),   # 900 MHz
        (2, 15.0, 2400),  # 2.4 GHz
        (2, 15.0, 5000),  # 5 GHz
    ]
    
    for walls, dist, freq in freq_test:
        try:
            features = pd.DataFrame([[walls, dist, freq]], 
                                  columns=['num_walls', 'distance', 'frequency'])
            pl = model.predict(features)[0]
            print(f"  {freq}MHz: {pl:.2f} dB")
        except Exception as e:
            print(f"  ❌ Erreur pour {freq}MHz: {e}")

def test_with_original_data(model):
    """
    Teste le modèle avec les données originales si disponibles
    """
    # Charger le dataset original
    df_original = pd.read_csv('pathloss_dataset.csv')
    print(f"\n=== TEST AVEC DONNEES ORIGINALES ===")
    print(f"Dataset chargé: {len(df_original)} échantillons")
    
    # Vérifier les colonnes
    print(f"Colonnes disponibles: {list(df_original.columns)}")
    
    # Prendre un échantillon pour test
    test_sample = df_original.sample(n=min(100, len(df_original)), random_state=42)
    
    # Features et target
    X_test = test_sample[['num_walls', 'distance', 'frequency']]
    y_true = test_sample['pathloss_dB']
    
    # Prédictions
    y_pred = model.predict(X_test)
    
    # Métriques
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"Performance sur échantillon test:")
    print(f"  MSE: {mse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R²: {r2:.4f}")
    
    return True
# Exécution des tests
if __name__ == "__main__":
    print("🔧 CHARGEMENT ET TEST DU MODELE 🔧")
    print("=" * 50)
    
    # Charger le modèle
    model = load_and_test_model()
    
    if model is not None:
        # Tester les prédictions
        test_cases, predictions = test_model_predictions(model)
        
        # Valider la logique
        validate_model_logic(test_cases, predictions)
        
        # Tester avec données originales
        test_with_original_data(model)
        
        print("\n✅ TESTS TERMINES AVEC SUCCES!")
        print("Le modèle est prêt à être utilisé.")
        
    else:
        print("\n❌ ECHEC DU CHARGEMENT DU MODELE")

🔧 CHARGEMENT ET TEST DU MODELE 🔧
✓ Modèle chargé avec succès!
Type de modèle: <class 'xgboost.sklearn.XGBRegressor'>
Impossible d'afficher les paramètres: 'XGBModel' object has no attribute 'feature_weights'

=== TESTS DE PREDICTION ===
Très proche, aucun mur (LOS)       : 53.09 dB
  Features: 0 murs, 5.0m, 2400MHz

Distance moyenne, 1 mur            : 61.64 dB
  Features: 1 murs, 10.0m, 2400MHz

Distance moyenne, 3 murs           : 79.78 dB
  Features: 3 murs, 15.0m, 2400MHz

Loin, aucun mur (LOS)              : 69.60 dB
  Features: 0 murs, 30.0m, 2400MHz

Loin, 5 murs (NLOS dense)          : 99.81 dB
  Features: 5 murs, 25.0m, 2400MHz

Distance moyenne, 2 murs, 5GHz     : 79.01 dB
  Features: 2 murs, 10.0m, 5000MHz

Proche, 1 mur, 900MHz              : 53.49 dB
  Features: 1 murs, 8.0m, 900MHz

Très très proche                   : 40.29 dB
  Features: 0 murs, 1.0m, 2400MHz

Très loin, nombreux murs           : 123.69 dB
  Features: 7 murs, 40.0m, 5800MHz

=== VALIDATION DE LA LOGIQUE

In [1]:
import pickle
import pandas as pd
import numpy as np

# Charger le modèle
model_filename = "xgboost_radio_propagation_model.pkl"

try:
    with open(model_filename, 'rb') as f:
        loaded_data = pickle.load(f)
    
    print("=== ANALYSE DU MODÈLE CHARGÉ ===")
    print(f"✓ Fichier chargé avec succès: {model_filename}")
    print(f"✓ Type de l'objet principal: {type(loaded_data)}")
    
    # Vérifier la structure
    if isinstance(loaded_data, dict):
        print("\n📊 STRUCTURE MODEL_INFO DÉTECTÉE:")
        print(f"✓ C'est un dictionnaire avec {len(loaded_data)} clés")
        
        print("\n🔑 Clés disponibles:")
        for i, key in enumerate(loaded_data.keys(), 1):
            if key == 'model':
                print(f"  {i}. {key}: {type(loaded_data[key])}")
            else:
                print(f"  {i}. {key}: {loaded_data[key]}")
        
        # Extraire le modèle
        model = loaded_data['model']
        feature_names = loaded_data['feature_names']
        metrics = loaded_data['metrics']
        
        print(f"\n🤖 TYPE DU MODÈLE:")
        print(f"✓ Type: {type(model)}")
        print(f"✓ Classe: {model.__class__.__name__}")
        print(f"✓ Module: {model.__class__.__module__}")
        
        print(f"\n📈 MÉTADONNÉES:")
        print(f"✓ Features requises: {feature_names}")
        print(f"✓ RMSE: {metrics['rmse']:.3f} dB")
        print(f"✓ R²: {metrics['r2_score']:.4f}")
        print(f"✓ MAE: {metrics['mae']:.3f} dB")
        
        # Test rapide
        print(f"\n🧪 TEST RAPIDE DE PRÉDICTION:")
        
        # Créer des données de test
        test_scenarios = [
            {"distance": 5, "numwall": 0, "etage": 0, "frequence": 2400, "desc": "5m, LOS, 2.4GHz"},
            {"distance": 20, "numwall": 2, "etage": 0, "frequence": 2400, "desc": "20m, 2 murs, 2.4GHz"},
            {"distance": 50, "numwall": 5, "etage": 1, "frequence": 5000, "desc": "50m, 5 murs, 1 étage, 5GHz"}
        ]
        
        print("Scénarios de test:")
        print("-" * 60)
        
        for scenario in test_scenarios:
            # Créer DataFrame avec les features dans le bon ordre
            test_input = pd.DataFrame({
                'distance': [scenario['distance']],
                'numwall': [scenario['numwall']],
                'etage': [scenario['etage']],
                'frequence': [scenario['frequence']]
            })
            
            # Prédiction
            prediction = model.predict(test_input)[0]
            
            print(f"{scenario['desc']:<30} → {prediction:.1f} dB")
        
        print("-" * 60)
        
        # Vérifier les paramètres du modèle
        print(f"\n⚙️ PARAMÈTRES DU MODÈLE:")
        params = model.get_params()
        important_params = ['n_estimators', 'max_depth', 'learning_rate', 'subsample']
        for param in important_params:
            if param in params:
                print(f"✓ {param}: {params[param]}")
        
        print(f"\n✅ RÉSUMÉ:")
        print(f"✓ Type: Dictionnaire model_info avec XGBRegressor")
        print(f"✓ Format: Structure complète avec métadonnées")
        print(f"✓ Features: 4 colonnes originales (distance, numwall, etage, frequence)")
        print(f"✓ Performance: R² = {metrics['r2_score']:.4f}")
        print(f"✓ État: Prêt à l'utilisation")
        
    else:
        # Si ce n'est pas un dictionnaire, c'est probablement juste le modèle
        print(f"✓ Type direct: {type(loaded_data)}")
        model = loaded_data
        
        # Test rapide direct
        test_input = pd.DataFrame({
            'distance': [10],
            'numwall': [1],
            'etage': [0],
            'frequence': [2400]
        })
        
        prediction = model.predict(test_input)[0]
        print(f"✓ Test de prédiction: {prediction:.1f} dB")
        
except FileNotFoundError:
    print(f"❌ Fichier non trouvé: {model_filename}")
    print("Vérifiez que le modèle a été sauvegardé.")
    
except Exception as e:
    print(f"❌ Erreur lors du chargement: {e}")
    import traceback
    traceback.print_exc()

=== ANALYSE DU MODÈLE CHARGÉ ===
✓ Fichier chargé avec succès: xgboost_radio_propagation_model.pkl
✓ Type de l'objet principal: <class 'dict'>

📊 STRUCTURE MODEL_INFO DÉTECTÉE:
✓ C'est un dictionnaire avec 10 clés

🔑 Clés disponibles:
  1. model: <class 'xgboost.sklearn.XGBRegressor'>
  2. feature_names: ['distance', 'numwall', 'etage', 'frequence']
  3. feature_importance: {'distance': np.float32(0.06101595), 'numwall': np.float32(0.32105938), 'etage': np.float32(0.6116421), 'frequence': np.float32(0.006282599)}
  4. hyperparameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.8, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'feature_weights': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_bin': None, 'max_cat_thre